In [338]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException

from datetime import datetime
import pandas as pd
from time import sleep
from tqdm import tqdm
from pprint import pprint
import re


In [339]:
browser = webdriver.Chrome()

In [409]:
browser.get('https://web.telegram.org/z/#-1321264581')

In [412]:
df = pd.DataFrame({
    'msg_date': [], 
    'msg_time': [], 
    'msg_text': [], 
    'msg_views': [], 
    'msg_id': []
})

In [413]:
start_all = datetime.now()

small_sleep = 5 # количество секунд, которые ждём перед каждым скролом вверх
long_sleep_batch = 10 # количество сколов, перед долгой паузой
long_sleep = 15 # количество секунд, которые ждём во время долгой паузы

before_long_sleep = long_sleep_batch # счётчик скролов
i = 0 # счётчик итераций
max_iter = 99999 # ограничитель итераций

is_scrapping = True # признак продолжения скраппинга


last_date = ''
while is_scrapping:
    i += 1
    if i >= max_iter:
        is_scrapping = False

    start = datetime.now()

    try: 
        container = browser.find_element(By.CLASS_NAME, 'messages-container')
        messages = container.find_elements(By.CLASS_NAME, 'message-date-group')    
    except NoSuchElementException as exc:
        print('NoSuchElementException')
        print(exc)
        is_scrapping = False
        messages = []
        
    for msg in tqdm(messages):
        msg_item = msg.find_element(By.CLASS_NAME, 'message-list-item')
        msg_item_id = msg_item.get_attribute('data-message-id')
        msg_date = msg.find_element(By.CLASS_NAME, 'sticky-date').text

        msg_content = msg.find_element(By.CLASS_NAME, 'text-content')
        if 'with-meta' in msg_content.get_attribute('class'):
            msg_text_lines = msg_content.text.split('\n')
            msg_time = msg_text_lines.pop() # время
            msg_views = msg_text_lines.pop() # просмотры
            msg_text = '\n'.join(msg_text_lines)
        else:
            msg_time = msg.find_element(By.CLASS_NAME, 'message-time').text
            msg_views = msg.find_element(By.CLASS_NAME, 'message-views').text
            msg_text = msg_content.text

        row = pd.DataFrame({
            'msg_date': [msg_date],
            'msg_time': [msg_time],
            'msg_text': [msg_text],
            'msg_views': [msg_views],
            'msg_id': [msg_item_id],
            })
        df = pd.concat([df, row], ignore_index=True)
        last_date = msg_date

    df.to_csv('data.csv')
    
    time = (datetime.now()-start).total_seconds()
    time_m = int(time//60)
    time_s = int(time - time_m*60)
    print(f'Итерация {i} >> {time_m}:{time_s}')
    print(f'Загруженная дата: {last_date} / Размер датасета: {df.shape[0]}')

    try: 
        before_long_sleep -= 1
        if before_long_sleep <= 0:
            print(f'Пауза {long_sleep}с...')
            before_long_sleep = long_sleep_batch
            for s in tqdm(range(long_sleep)):
                sleep(1)
        else:
            sleep(small_sleep)

        back_trigger = browser.find_element(By.CLASS_NAME, 'backwards-trigger')
        actions = ActionChains(browser)
        actions.move_to_element(back_trigger).perform()    

    except NoSuchElementException as exc:
        print('NoSuchElementException')
        print(exc)
        is_scrapping = False

time_all = (datetime.now()-start_all).total_seconds()
time_m = int(time_all//60)
time_s = int(time_all - time_m*60)
print(f'Общее время >> {time_m}:{time_s}')
    

100%|██████████| 16/16 [00:01<00:00, 10.65it/s]


Итерация 1 >> 0:1
Загруженная дата: Today / Размер датасета: 16


100%|██████████| 37/37 [00:02<00:00, 14.09it/s]


Итерация 2 >> 0:2
Загруженная дата: Today / Размер датасета: 53


100%|██████████| 48/48 [00:03<00:00, 12.47it/s]


...

Итерация 29 >> 0:8
Загруженная дата: July 19, 2020 / Размер датасета: 1641


100%|██████████| 55/55 [00:08<00:00,  6.71it/s]


Итерация 30 >> 0:8
Загруженная дата: May 21, 2020 / Размер датасета: 1696
Пауза 15с...


  0%|          | 0/33 [00:00<?, ?it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".text-content"}
  (Session info: chrome=108.0.5359.124)
Stacktrace:
0   chromedriver                        0x000000010e25af38 chromedriver + 4910904
1   chromedriver                        0x000000010e1daa03 chromedriver + 4385283
2   chromedriver                        0x000000010de1f747 chromedriver + 472903
3   chromedriver                        0x000000010de6434c chromedriver + 754508
4   chromedriver                        0x000000010de645a1 chromedriver + 755105
5   chromedriver                        0x000000010de56216 chromedriver + 696854
6   chromedriver                        0x000000010de8a13d chromedriver + 909629
7   chromedriver                        0x000000010de560ed chromedriver + 696557
8   chromedriver                        0x000000010de8a2ce chromedriver + 910030
9   chromedriver                        0x000000010dea528e chromedriver + 1020558
10  ch

In [414]:
df.drop_duplicates(inplace=True)
df.to_csv('data.csv')

894